In [1]:
#dataset   https://www.kaggle.com/c/fake-news/data?select=train.csv

import os
os.getcwd()
os.chdir('D:\Cse\dataset')

In [2]:
import pandas as pd
df = pd.read_csv('fake_news.csv')

In [3]:
df.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [4]:
#drop nan values
df = df.dropna()

In [5]:
#separating output "label" to y
X = df.drop('label', axis=1)
y = df['label']

In [6]:
import tensorflow as tf

In [7]:
tf.__version__

'2.0.0'

In [8]:
from tensorflow.keras.layers import Embedding
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.layers import LSTM 
from tensorflow.keras.layers import Dense


In [9]:
#vocabulary size
voc_size = 5000

In [10]:
messages = X.copy()

In [11]:
messages.reset_index(inplace=True)


In [12]:
import nltk
import re
from nltk.corpus import stopwords

In [13]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Ratul\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [15]:
#data preprocessing

from nltk.stem.porter import PorterStemmer
ps = PorterStemmer()
corpus = []
for i in range(0, len(messages)):
    
    review = re.sub('[^a-zA-z]', ' ', messages['title'][i])
    review = review.lower()
    review = review.split()
    
    review = [ps.stem(word) for word in review if not word in stopwords.words('english')]
    review = ' '.join(review)
    corpus.append(review)

    

In [16]:
onehot_rep = [one_hot(words,voc_size) for words in corpus]
onehot_rep

[[4589, 1696, 3941, 4835, 4788, 2309, 2738, 1353, 4455, 2960],
 [1371, 1713, 1147, 2420, 4871, 1491, 3807],
 [2593, 600, 3802, 2107],
 [936, 4932, 1205, 801, 2521, 3812],
 [853, 4871, 2131, 544, 4286, 762, 4871, 2875, 930, 2075],
 [907,
  1179,
  4324,
  2814,
  3476,
  1968,
  766,
  2000,
  4186,
  2726,
  2584,
  2540,
  3818,
  3632,
  3807],
 [3794, 1546, 2989, 1181, 1530, 1251, 1099, 2590, 199, 3243, 3845],
 [2001, 739, 67, 2771, 2883, 2712, 1968, 507, 199, 3243, 3845],
 [417, 584, 1560, 2765, 141, 3932, 4772, 1223, 1968, 3799],
 [2931, 1362, 2382, 3327, 3690, 4496, 3243, 699],
 [129, 924, 3872, 37, 267, 4246, 3099, 3784, 354, 3625, 325],
 [801, 4332, 4788, 3932, 1968, 2883],
 [3205, 3840, 2422, 1086, 1013, 3751, 15, 1004, 4051],
 [270, 2207, 1176, 1527, 1250, 3466, 79, 199, 3243, 3845],
 [2471, 3404, 4825, 2955, 204, 199, 3243, 3845],
 [1081, 3251, 4921, 3470, 466, 2445, 2806, 579, 1360, 2108],
 [3041, 1895, 1713],
 [4640, 2720, 4603, 558, 1968, 241, 2571, 3807],
 [4082, 4427, 1

In [18]:
sent_length = 20
embedded_docs = pad_sequences(onehot_rep,padding='pre', maxlen =sent_length)
print(embedded_docs)

[[   0    0    0 ... 1353 4455 2960]
 [   0    0    0 ... 4871 1491 3807]
 [   0    0    0 ...  600 3802 2107]
 ...
 [   0    0    0 ...  199 3243 3845]
 [   0    0    0 ... 1361 2942 1293]
 [   0    0    0 ... 2386  100 1032]]


In [19]:
len(embedded_docs)

18285

In [21]:
#creating model
embedding_vector_features = 40
model = Sequential()
model.add(Embedding(voc_size,embedding_vector_features, input_length = sent_length))
model.add(LSTM(100))
model.add(Dense(1,activation = 'sigmoid'))
model.compile(loss = 'binary_crossentropy', optimizer = 'adam', metrics = ['accuracy'])
print(model.summary())

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 20, 40)            200000    
_________________________________________________________________
lstm (LSTM)                  (None, 100)               56400     
_________________________________________________________________
dense (Dense)                (None, 1)                 101       
Total params: 256,501
Trainable params: 256,501
Non-trainable params: 0
_________________________________________________________________
None


In [22]:
len(embedded_docs),y.shape

(18285, (18285,))

In [25]:
import numpy as np
X_final = np.array(embedded_docs)
y_final = np.array(y)

In [26]:
X_final.shape, y_final.shape

((18285, 20), (18285,))

In [28]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_final, y_final, test_size = 0.33, random_state = 42)

In [30]:
#Model Training

model.fit(X_final, y_final, validation_data = (X_test, y_test), epochs = 10, batch_size = 64)

Train on 18285 samples, validate on 6035 samples
Epoch 1/10
18285/18285 [==============================] - 37s 2ms/sample - loss: 0.2809 - accuracy: 0.8713 - val_loss: 0.1322 - val_accuracy: 0.9511
Epoch 2/10
18285/18285 [==============================] - 28s 2ms/sample - loss: 0.1413 - accuracy: 0.9443 - val_loss: 0.0951 - val_accuracy: 0.9679A: 10s - ETA: 9s - loss: 0 - ETA: 4s - loss: 0.1380 - accura - ETA: 2s - loss: 0.1392 - accura - ETA: 2s - loss: 0.1393 - accuracy: 0.94 - ETA: 2s - loss: 0.1393 - accuracy: 0.94 - ETA: 2s - loss: 0.1393 - accuracy: 0. - ETA: 1s - loss: 0.1395 - accuracy: 0. - ETA: 1s - loss: 0.1395 - ac - ETA: 1s - loss: 0.1409 - accuracy - ETA: 0s - loss: 0.1410 - accura - ETA: 0s - loss: 0.1412 - accuracy: 0.94 - ETA: 0s - loss: 0.1411 - accuracy: 0.94 - ETA: 0s - loss: 0.1412 - accuracy: 0. - ETA: 0s - loss: 0.1410 - accuracy: 0.94 - ETA: 0s - loss: 0.1409 - accuracy: 0.
Epoch 3/10
18285/18285 [==============================] - 29s 2ms/sample - loss: 0.1055 -

In [32]:
#performance matrics and Accuracy

y_pred = model.predict_classes(X_test)
from sklearn.metrics import confusion_matrix
confusion_matrix(y_test, y_pred)

array([[3409,   10],
       [   2, 2614]], dtype=int64)

In [33]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test, y_pred)

0.9980115990057995